# Heartbeat Detection

## Imports

In [2]:
import copy
import wave
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation, Dropout
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
import math

## Load heartbeat sounds

In [18]:
# Classes to handle, ordered by label
CLASSES = ['artifact','arrythmia','normal', 'noise']
DATASET_DIR = Path('dataset_heartbeat')

x_train = []
y_train = []
x_test = []
y_test = []

for recording in DATASET_DIR.glob(f'**/*.wav'):
    if not recording.parent.name in CLASSES: # Ignore unused classes
        continue
    label = CLASSES.index(recording.parent.name) # Assign class number
    
    with wave.open(str(recording)) as f: # Read wave file
        data = np.frombuffer(f.readframes(f.getnframes()), dtype=np.int16).copy() # As 16-bit signed integer
        
    data = data.astype(np.float32) # Convert to 32-bit floating-point
    data.resize((16000, 1)) # Resize to 1s (16kHz) with zero-padding, 1 channel
    
    x_train.append(data)
    y_train.append(label)

x_train = np.array(x_train)
y_train = to_categorical(np.array(y_train))

perms = np.random.permutation(len(y_train))[0:math.ceil(len(y_train)*0.2)]
x_test = x_train[perms]
y_test = y_train[perms]

x_train = np.delete(x_train, perms, axis=0)
y_train = np.delete(y_train, perms, axis=0)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(735, 16000, 1)
(184, 16000, 1)
(735, 4)
(184, 4)


## Combine set A and B

In [19]:
print ("")
print ("training data shape: ", x_train.shape)
print ("training label shape: ", y_train.shape)
print ("")
print ("test data shape: ", x_test.shape)
print ("test label shape: ", y_test.shape)


training data shape:  (735, 16000, 1)
training label shape:  (735, 4)

test data shape:  (184, 16000, 1)
test label shape:  (184, 4)


## Prepare for inference with fixed-point Q7.9 samples by scaling input data accordingly

In [20]:
FIXED_POINT = 9
x_train /= 2**FIXED_POINT
x_test  /= 2**FIXED_POINT

## Build model *not* M5

In [70]:
# Define hyperparameters
num_classes = len(CLASSES) # Number of sound classes
num_filters = 2 # Number of filters for CNN layers
num_units = 128 # Number of units for RNN layers
num_heads = 8 # Number of heads for attention layers
dropout_rate = 0.1 # Dropout rate for regularization

model = Sequential()
model.add(Input(shape=(16000, 1)))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(2, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(4, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(8, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(16, kernel_size=3, activation='relu'))
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(units=num_classes, activation='softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-6)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 max_pooling1d_141 (MaxPooli  (None, 4000, 1)          0         
 ng1D)                                                           
                                                                 
 conv1d_154 (Conv1D)         (None, 3998, 2)           8         
                                                                 
 max_pooling1d_142 (MaxPooli  (None, 999, 2)           0         
 ng1D)                                                           
                                                                 
 conv1d_155 (Conv1D)         (None, 997, 4)            28        
                                                                 
 max_pooling1d_143 (MaxPooli  (None, 249, 4)           0         
 ng1D)                                                           
                                                     

## Train model

In [71]:
model.fit(x_train, y_train, epochs=50, batch_size=10, validation_data=(x_test, y_test))

Epoch 1/50
74/74 [==============================] - 3s 20ms/step - loss: 1.3906 - categorical_accuracy: 0.1619 - val_loss: 1.3823 - val_categorical_accuracy: 0.2228
Epoch 2/50
74/74 [==============================] - 1s 17ms/step - loss: 1.3692 - categorical_accuracy: 0.2789 - val_loss: 1.3561 - val_categorical_accuracy: 0.3641
Epoch 3/50
74/74 [==============================] - 1s 17ms/step - loss: 1.3497 - categorical_accuracy: 0.3293 - val_loss: 1.3333 - val_categorical_accuracy: 0.4076
Epoch 4/50
74/74 [==============================] - 1s 17ms/step - loss: 1.3330 - categorical_accuracy: 0.3578 - val_loss: 1.3116 - val_categorical_accuracy: 0.4620
Epoch 5/50
74/74 [==============================] - 1s 17ms/step - loss: 1.3172 - categorical_accuracy: 0.3837 - val_loss: 1.2921 - val_categorical_accuracy: 0.5054
Epoch 6/50
74/74 [==============================] - 1s 16ms/step - loss: 1.3013 - categorical_accuracy: 0.4177 - val_loss: 1.2706 - val_categorical_accuracy: 0.5380
Epoch 7/50

74/74 [==============================] - 1s 17ms/step - loss: 0.8386 - categorical_accuracy: 0.7116 - val_loss: 0.8091 - val_categorical_accuracy: 0.7826


## Evaluate model on test dataset

In [72]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

6/6 - 0s - loss: 0.8091 - categorical_accuracy: 0.7826 - 89ms/epoch - 15ms/step
6/6 [==============================] - 0s 18ms/step
tf.Tensor(
[[ 0  1  5  1]
 [ 0  4 25  0]
 [ 0  1 77  2]
 [ 0  1  4 63]], shape=(4, 4), dtype=int32)


## Save trained model

In [73]:
model.save('heartbeat_78percent.h5')

np.savetxt('x_test.csv', x_test.reshape((x_test.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test.csv', y_test, delimiter=',', fmt='%s')

## Remove SoftMax layer

In [74]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [57]:
%pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='bitbucket.org', port=443): Max retries exceeded with url: /edge-team-leat/microai_public/get/6adfbcb347d3.zip (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x0000022A67D76AD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Generate C code for the trained model with 16-bit fixed-point representation

In [75]:
res = kerascnn2c.Converter(output_path=Path('gsc_output_fixed'),
                           fixed_point=FIXED_POINT, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('full_model.h', 'w') as f:
    f.write(res)

———————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                           | Layer                            | Outputs                         
———————————————————————————————————————————————————————————————————————————————————————————————————————
                                 | input_21                         | max_pooling1d_141               
-------------------------------------------------------------------------------------------------------
input_21                         | max_pooling1d_141                | conv1d_154                      
-------------------------------------------------------------------------------------------------------
max_pooling1d_141                | conv1d_154                       | max_pooling1d_142               
-------------------------------------------------------------------------------------------------------
conv1d_154                       | max_pooling1d_142                

## Compile the 16-bit fixed-point C code for x86 and evaluate on small dataset

In [76]:
!g++ -Wall -Wextra -pedantic -Ofast -o gsc_fixed -Igsc_output_fixed/ gsc_output_fixed/model.c main.cpp 
!./gsc_fixed x_test.csv y_test.csv

gsc_output_fixed/model.c: In function 'void cnn(const number_t (*)[16000], number_t*)':
gsc_output_fixed/model.c:178:18: warning: left operand of comma operator has no effect [-Wunused-value]
  178 |     activations1.max_pooling1d_145_output,
      |     ~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~
'.' is not recognized as an internal or external command,
operable program or batch file.
